In [32]:
from tkinter import *
import matplotlib.pyplot as plt
#bigger fonts
import matplotlib
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rcParams.update({'font.size': 20})

matplotlib.use("TkAgg")

import pandas as pd
import numpy as np
import scipy.stats as ss

# Reading properties

In [11]:
T = pd.read_csv('../TOST_data/average_properties.txt',
            sep='\t',index_col=0,na_values=-99,
            dtype={'WEIGHT_GRAMS': float,
                   'GEST_AGE_DAYS':float,
                   'DELTA_SEC_MEASURE':float,
                   'MED_SPO2':float,
                   'MED_PI':float,
                   'MED_PR':float})

In [12]:
# GESTATIONAL AGE BINNING
slots = [0.,38*7,39*7,40*7,41*7,500.]
sel = []
for i in range(len(slots)-1):
    sel.append((T['GEST_AGE_DAYS']>= slots[i])&(T['GEST_AGE_DAYS']< slots[i+1]))

In [36]:
# TIME SINCE BIRTH BINNING
slots_hour = [0.,10,20,30,40,100.]
DELTA_HOURS = np.array(T['DELTA_SEC_MEASURE'])/60./60.
sel_delta_t = []
for i in range(len(slots_hour)-1):
    sel_delta_t.append((DELTA_HOURS>= slots_hour[i])&(DELTA_HOURS< slots_hour[i+1]))

# Functions

In [13]:
def plot_hist_split(prop='MED_PR',select=((T['MED_PR']>10)& (T['PHN']==0) & (T['FOOT']==1)),running_sel=sel,slots=np.array(slots)/7.):
    fig = plt.figure(figsize=(10,8))
    for i in range(len(running_sel)):
        data = T[prop][running_sel[i] & select]  
        plt.hist(data,bins=20,histtype='step',color='C%i'%i,lw=3,label='[%.1f,%.1f] mean=%.1f'%(slots[i],slots[i+1],np.mean(data)))
        plt.axvline(np.mean(data),color='C%i'%i,ls='dashed',lw=3)
    plt.xlabel(prop)
    plt.ylabel('counts')
    plt.legend(fontsize='x-small')
    #plt.title('NO PHN - ONLY FOOT')
    return fig

def plot_graph_split(prop='MED_PR',select=((T['MED_PR']>10)& (T['PHN']==0) & (T['FOOT']==1)),running_sel=sel,slots=np.array(slots)/7.,running_prop='GESTATIONAL_AGE',txt_y=100,txt_x=38,conversion=1/7.):
    fig = plt.figure(figsize=(10,8))
    y = []
    x = []
    xerr = []
    yerr = []

    for i in range(len(running_sel)):
        tot_sel = running_sel[i] & select
        data = T[prop][tot_sel]  
        
        if i==0:
            data_first = data
        elif i ==len(running_sel)-1:
            data_last = data
        
        y.append(np.mean(data))
        
        
        x.append(np.mean(T[running_prop][tot_sel])*conversion)
        #xerr.append(np.std(T[running_prop][tot_sel]/7.)/np.sqrt(len(data)))
        
        
        yerr.append(np.std(data))#/np.sqrt(len(data)) )
        #yerr.append(ss.iqr(data) )

        
        #plt.hist(data,bins=20,histtype='step',color='C%i'%i,lw=3,label='[%.1f,%.1f] mean=%.1f'%(slots[i],slots[i+1],np.mean(data)))
        #plt.axvline(np.mean(data),color='C%i'%i,ls='dashed',lw=3)
        print(len(data))
        #print(len(T[running_prop][tot_sel]))
    x = np.array(x)
    y = np.array(y)
    #xerr = np.array(xerr)
    yerr = np.array(yerr)
    
    #98,140
    plt.text(txt_x,txt_y,'difference extremes = %.2f, p-value %.6f'%(np.mean(data_first)-np.mean(data_last),ss.ttest_ind(data_first,data_last)[1]))
    
    #plt.errorbar(x,y,xerr=xerr,yerr=yerr,color='k',lw=5)
    plt.errorbar(x,y,yerr=yerr,color='k',lw=5)
    plt.plot(x,y,".",markersize=30,color='k')

    plt.xlabel(running_prop)
    plt.ylabel(prop)
    #plt.legend(fontsize='x-small')
    #plt.title('NO PHN - ONLY FOOT')
    return fig



In [38]:
def Update_lbl():
    lbl_prop.configure(text=str(prop.get()))
    
def plot_histo():
    plot_hist_split(str(prop.get()),select=(T[str(prop.get())]>10)& (T['PHN']==0) & (T['WRIST']==1),running_sel=sel)
    plt.xlabel(str(prop.get()))
    plt.title('NO PHN - ONLY FOOT')
    plt.show()
    
    
def plot_graph():
    plot_graph_split(str(prop.get()),select=(T[str(prop.get())]>10)& (T['PHN']==0) & (T['WRIST']==1),running_prop='GEST_AGE_DAYS',running_sel=sel,txt_y=136)
    plt.xlabel('GESTATIONAL AGE WEEKS')
    plt.title('NO PHN - ONLY WRIST')
    plt.show()

    
def stat():
    for i in range(len(slots_hour)-1):
        print('MEAN VALUES %i < delta_t <  %i [hours]'%(slots_hour[i],slots_hour[i+1]))
        print('##################################')
        print(T[sel_delta_t[i]].mean())
        print('\n')

window = Tk()
window.title("Prakash project NHS")
window.geometry('950x600')
lbl = Label(window,text='Select property to be plotted:',font=("Arial Bold", 15))#.grid(column=0,row=0)
lbl.grid(row=0,column=0)
prop = StringVar()
prop.set('MED_PR')
lbl_prop = Label(window, text= str(prop.get()))
lbl_prop.grid(row=0,column=4)
r1 = Radiobutton(window, text="Pulse Rate", variable=prop, value='MED_PR',indicatoron=0,command=Update_lbl).grid(row=0,column=1)
r2 = Radiobutton(window, text="Oxigen Saturation", variable=prop, value='MED_SPO2',indicatoron=0,command=Update_lbl).grid(row=0,column=2)
r3 = Radiobutton(window, text="Perfusion index", variable=prop, value='MED_PI',indicatoron=0,command=Update_lbl).grid(row=0,column=3)

btn_hist = Button(window,text='Histogram in bins',bg='blue',fg='white',command=plot_histo)
btn_hist.grid(column=0,row=1)

btn_plot = Button(window,text='Plot in bins',bg='red',fg='white',command=plot_graph)
btn_plot.grid(column=0,row=2)

btn_stat = Button(window,text='stats',bg='green',fg='white',command=stat)
btn_stat.grid(column=0,row=3)



In [ ]:
window.mainloop()

MEAN VALUES 0 < delta_t <  10 [hours]
##################################
WEIGHT_GRAMS          3426.857143
GEST_AGE_DAYS          277.095238
DELTA_SEC_MEASURE    28147.547619
MED_SPO2                96.321429
MED_PR                 115.434524
MED_PI                   1.136250
PHN                      0.000000
WRIST                    0.500000
FOOT                     0.440476
dtype: float64


MEAN VALUES 10 < delta_t <  20 [hours]
##################################
WEIGHT_GRAMS          3348.016611
GEST_AGE_DAYS          277.132890
DELTA_SEC_MEASURE    54513.378738
MED_SPO2                96.365449
MED_PR                 121.943522
MED_PI                   1.169534
PHN                      0.013289
WRIST                    0.485050
FOOT                     0.481728
dtype: float64


MEAN VALUES 20 < delta_t <  30 [hours]
##################################
WEIGHT_GRAMS          3256.162602
GEST_AGE_DAYS          275.512195
DELTA_SEC_MEASURE    82956.422764
MED_SPO2                94.9878